In [1]:
import torchvision
import torch

In [2]:
torchvision.__version__

'0.15.2'

In [3]:
torch.__version__

'2.0.1'

In [4]:
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Lambda
transform = Compose([
    ToTensor(),
    # Lambda(lambda image: image / 255),
    Lambda(lambda image: image.view(784))
])
data_train = MNIST(root="./",download=True,train=True,transform=transform)
data_test = MNIST(root="./",download=True,train=False,transform=transform)

100%|███████████████████████████████████████████████████████████████████| 9912422/9912422 [00:01<00:00, 6116516.39it/s]


Extracting ./MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST\raw



100%|██████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 11252735.14it/s]

Extracting ./MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST\raw


100%|███████████████████████████████████████████████████████████████████| 1648877/1648877 [00:00<00:00, 3911771.87it/s]


Extracting ./MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST\raw



100%|█████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 4556452.71it/s]

Extracting ./MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST\raw



In [5]:
data_train[0][0].shape

torch.Size([784])

In [6]:
data_train[0][1]

5

In [7]:
from torch import nn, optim
import torch
def get_default_device():
    return torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

def to_t(tensor,device=get_default_device()):
    return tensor.to(device)

class MNISTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(784,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
        self.loss = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.parameters())
        self.to(get_default_device())
    def forward(self,X):
        return self.layers(X)
    
    def predict(self,X):
        with torch.no_grad():
            return torch.argmax(self.forward(X),axis=-1)
    def fit(self,X,Y):
        self.optimizer.zero_grad()
        y_pred = self.forward(X)
        loss = self.loss(y_pred,Y)
        loss.backward()
        self.optimizer.step()
        return loss.item()

In [8]:
mnist_model = MNISTModel()

In [9]:
from torch.utils.data import DataLoader
BATCH_SIZE = 16
dataloader_train = DataLoader(data_train,batch_size=BATCH_SIZE,shuffle=True)
dataloader_test = DataLoader(data_test,batch_size=BATCH_SIZE,shuffle=True)

In [10]:
from tqdm import tqdm
EPOCHS = 5
for i in range(EPOCHS):
    total_loss = 0
    for r_xs, r_ys in tqdm(dataloader_train,desc=f"FITTING EPOCH {i}"):
        xs, ys = to_t(r_xs), to_t(r_ys)
        total_loss += mnist_model.fit(xs,ys)
    total_loss /= len(dataloader_train)
    print(f"EPOCH {i}: {total_loss:.4f}")

FITTING EPOCH 0: 100%|████████████████████████████████████████████████████████████| 3750/3750 [00:26<00:00, 144.07it/s]


EPOCH 0: 0.1974


FITTING EPOCH 1: 100%|████████████████████████████████████████████████████████████| 3750/3750 [00:20<00:00, 180.01it/s]


EPOCH 1: 0.0893


FITTING EPOCH 2: 100%|████████████████████████████████████████████████████████████| 3750/3750 [00:21<00:00, 177.02it/s]


EPOCH 2: 0.0638


FITTING EPOCH 3: 100%|████████████████████████████████████████████████████████████| 3750/3750 [00:20<00:00, 179.34it/s]


EPOCH 3: 0.0493


FITTING EPOCH 4: 100%|████████████████████████████████████████████████████████████| 3750/3750 [00:20<00:00, 180.65it/s]

EPOCH 4: 0.0426


In [11]:
correct = 0
for r_xs, r_ys in dataloader_test:
    # print(mnist_model.predict(xs))
    xs, ys = to_t(r_xs), to_t(r_ys)
    y_pred = mnist_model.predict(xs)
    correct += (ys == y_pred).sum()
acc = correct / (len(dataloader_test) * BATCH_SIZE)
print(f"ACCURACY: {acc}")

ACCURACY: 0.9787999987602234


In [12]:
len(dataloader_test)

625

In [13]:
625*16

10000